In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
# Load the dataset
ds = xr.open_dataset("madhuri_hodogram_60_70N_30_05W.nc")

# Explore variable names
print(ds)

In [ ]:
import matplotlib.pyplot as plt

# Select the region and pressure levels with correct dimension names
pres_range = ds['pressure_level'].sel(pressure_level=slice(850, 200))  # descending pressure levels

lat_range = slice(70, 60)    # latitude decreases, so slice from high to low
lon_range = slice(-30, -5)   # longitude increases, so slice as is

# Select u and v for the first time step and selected ranges
u = ds['u'].sel(valid_time=ds.valid_time[0],
                 pressure_level=pres_range,
                 latitude=lat_range,
                 longitude=lon_range)

v = ds['v'].sel(valid_time=ds.valid_time[0],
                 pressure_level=pres_range,
                 latitude=lat_range,
                 longitude=lon_range)

print(u.dims)  # Should show ('pressure_level', 'latitude', 'longitude')
print(v.dims)

# Average over latitude and longitude to get mean vertical profile
u_avg = u.mean(dim=['latitude', 'longitude'])
v_avg = v.mean(dim=['latitude', 'longitude'])

# Ensure pressure levels are sorted descending for plotting
u_avg = u_avg.sortby('pressure_level', ascending=False)
v_avg = v_avg.sortby('pressure_level', ascending=False)

# Plot the hodograph
plt.figure(figsize=(6, 6))
plt.plot(u_avg, v_avg, marker='o', linestyle='-')

# Annotate pressure levels on the plot
for i, p in enumerate(u_avg['pressure_level'].values):
    plt.text(u_avg[i].values, v_avg[i].values, f"{int(p)} hPa", fontsize=8, ha='right')

plt.xlabel("U wind (m/s)")
plt.ylabel("V wind (m/s)")
plt.title("Hodograph: Lat 60–70°, Lon −30 to −5°, 850–200 hPa")
plt.grid(True)
plt.axis('equal')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from matplotlib.patches import Ellipse
import matplotlib.cm as cm

# Coordinates and pressure range
lat_point = 62.0
lon_point = -14.0
pres_range = slice(400, 200)  # descending pressure levels

# Select times 10 to 13 (indices 9 to 12)
times = ds.valid_time[9:13]  # length 4
num_steps = len(times)

plt.figure(figsize=(8, 8))

colors = cm.viridis(np.linspace(0, 1, num_steps))
markers = ['o', 's', 'D', '^']
linestyles = ['-', '--', ':', '-.']

# To hold all perturbation points from all times for combined ellipse
all_perturbations = []

for i, t in enumerate(times):
    # Extract u and v profiles for time t
    u_profile = ds['u'].sel(valid_time=t,
                            latitude=lat_point,
                            longitude=lon_point,
                            pressure_level=pres_range)
    v_profile = ds['v'].sel(valid_time=t,
                            latitude=lat_point,
                            longitude=lon_point,
                            pressure_level=pres_range)

    # Compute perturbations (de-meaned)
    u_mean = u_profile.mean().item()
    v_mean = v_profile.mean().item()
    u_pert = u_profile - u_mean
    v_pert = v_profile - v_mean

    # Stack into 2D array for PCA and plotting
    X = np.column_stack([u_pert.values, v_pert.values])

    # Append to all perturbations
    all_perturbations.append(X)

    # Plot points and connecting line for this time step
    plt.plot(X[:, 0], X[:, 1], marker=markers[i], linestyle=linestyles[i], color=colors[i],
             label=f'Time step {i + 10} ({str(t.values)[:19]})')

# Combine all perturbations for a single PCA ellipse
all_perturbations = np.vstack(all_perturbations)

# Fit ellipse on combined data
pca = PCA(n_components=2)
pca.fit(all_perturbations)
center = pca.mean_
axes = 2 * np.sqrt(pca.explained_variance_)  # ellipse radii
angle = np.degrees(np.arctan2(*pca.components_[0][::-1]))

# Plot the combined ellipse
ellipse = Ellipse(xy=center, width=axes[0], height=axes[1],
                  angle=angle, edgecolor='black', fc='None', lw=2, label='Combined Ellipse')
plt.gca().add_patch(ellipse)

plt.axhline(0, color='gray', linestyle='--')
plt.axvline(0, color='gray', linestyle='--')

plt.xlabel("U' (m/s)")
plt.ylabel("V' (m/s)")
plt.title("Perturbation Hodograph with Single Combined Ellipse\n(62°N, −14.0°), 300–200 hPa")
plt.grid(True)
plt.axis('equal')
plt.legend(loc='best')
plt.tight_layout()
plt.show()
